# Language Exploration Project

Most language detection models are based on dictionaries of known words, with models that detect high level linguistic features like grammatical structure used to increase accuracy.  However, I would like to explore creating a model that is not dependent on a dictionary or knowing what language\(s\) words belong to ahead of time. My purpose is to explore language detection modeling with features that are character\-based, or based on patterns in relationships between characters in different languages.

## Goals

- Explore what lower level, character\-based structures exist within languages, and how they differ between languages.

- Determine the importance of various character\-based features in language detection, and their viability for continued efforts towards creating robust language detection systems that do not rely on dictionaries.

### Modeling Task

The modeling task I will be performing in order to explore these themes is the basic language detection task. It is a supervised machine learning classification problem with language classes as targets, and various features of text, which I will engineer, as the predictors.



## Original Data

I chose two sources of data that had a large variety of the same languages, but differed in the length of the text in order to get a larger variety of text lengths. Both data sources contain language labels that are equivalent to ISO 693\-3 codes.

### Wili 2018 Dataset

WiLI\-2018 is a Wikipedia language identification benchmark dataset. It contains paragraphs from different language Wikipedias. There are 235 language classes. For each language class there are 1,000 paragraphs, totaling 235,000 paragraphs. The size of the unzipped CSV file is 131.6 MB. The data was uploaded to Kaggle for convenience, but was originally uploaded to [zendo.org](http://Zendo.org) along with an [accompanying paper](https://arxiv.org/pdf/1801.07779v1.pdf). I retrieved the data through Kaggle, but I provide the original citation, as well.

#### Kaggle Citation

Menon, S. \(2020, August\). Wili 2018, Version 2. Retrieved February, 2023 from [https://www.kaggle.com/datasets/sharansmenon/wili\-2018﻿/versions/2](<https://www.kaggle.com/datasets/sharansmenon/wili-2018﻿/versions/2>).

#### Zenodo Citation

Thoma, M. \(2018\). WiLI\-2018 \- Wikipedia Language Identification database \(1.0.0\) \[Data set\]. Zenodo. https://doi.org/10.5281/zenodo.841984 

### Big Language Detection Dataset

The Big Language Detection Dataset contains sentences in 404 languages. There are 10,330,761 total sentences, which are not evenly distributed between the 404 language classes. The size of the unzipped CSV file is 580 MB. I also retrieved this data from Kaggle.

#### Citation

Chazzer. \(2022, September\). Big Language Detection Dataset, Version . Retrieved February, 2023 from [https://www.kaggle.com/datasets/chazzer/big\-language\-detection\-dataset/versions/3](https://www.kaggle.com/datasets/chazzer/big-language-detection-dataset/versions/3).



## Description of Final Dataset

After data cleaning and feature creation, my [final dataset](data/main_df.csv) had 85,999 rows with 51 columns. 43 languages are represented. The size of the file is 31 MB.

Columns include:

- 1 column for the target: ISO 693\-3 codes representing a language class.
- 1 column of text: The original text, retained for any future feature creation or for reference.
- 3 columns: Statistical features engineered from the text, floating\-point numerical data.
- 46 columns: N\-gram features engineered from the text. Integer counts of n\-gram frequencies.



## Data Cleaning

During the EDA process, I performed the following data cleaning steps:

- I standardized column names between the two original datasets \([see here](./EDA.ipynb#Initial-Data-Cleaning)\).
- I drop the unnecessary id column from the bld dataset.
- I stripped "\\n" from the language codes in the wili2018 dataset.
- I removed "junk data" that has no alphabetic characters \([see here](./EDA.ipynb#Removing-Junk-Text)\).
- I removed rows where the text was too short \([see here](./EDA.ipynb#Imposing-a-Minimum-Character-Length)\). I chose a threshold of 10 characters because at this length, the data starts to show sentences with at least two words.
- The datasets had different sets of languages, so I chose to remove languages that did not intersect between the two datasets \([see here](./EDA.ipynb#Intersecting-Languages)\).
- I limited the datasets to languages with at least 96% Latin\-based characters in the full corpus of the language's text for each dataset.  \([See here for methodology on determining percentage](./EDA.ipynb#Limiting-to-Languages-with-Latin-Based-Alphabets)\).
- I removed constructed languages, pidgins, and creoles using a list built manually from [Wiktionary](https://en.wiktionary.org/wiki/Appendix:ISO_639-3_codes/Others) and [Wikipedia](https://en.wikipedia.org/wiki/Codes_for_constructed_languages) lists, stored in [misc\_languages.json](./misc_languages.json) \([see here](./EDA.ipynb#Removing-Constructed-Languages,-Pidgins-and-Creoles)\).
- In order to make n\-gram features efficiently, I limited the set of characters to Latin based characters by replacing all non Latin based characters with a special character. I used a full block because it is rarely used, it works nicely with my other features, and it is easy to see it when I examine the text \([see here](./EDA.ipynb#Replacing-Non-Latin-Characters)\).
- To evenly represent each language, I limited the datasets to languages with at least 1,000 data points in each dataset. Later, I sampled 1,000 data points from each language in each dataset to merge into one dataset \([see here](./EDA.ipynb#Representing-Each-Language-Equally)\).
- After examining the feature data, I eliminated outlier data that was junk text \([see here](./EDA.ipynb#Removing-Outliers)\).



## Exploratory Data Analysis

### Basic Dataset Analysis

I did some [basic statistics](EDA.ipynb#Data-Examination) on the languages and texts in my two original datasets. The following image shows the results for the bld data.

<img src="images/bld_statistics.png"   width="1035px"  height="467.783px"  style="object-fit:cover"/>

Qualitatively, the wili2018 texts look like what I would expect from Wikipedia \- relatively formal writing with a lot of proper nouns and dates represented. The bld texts appear to be somewhat more conversational.

It looks like there are a couple of problems with the bld dataset:

1. There is an uneven representation of languages, with some of the languages having only 1 sentence. My goal is to evenly represent the target languages, so I will want to add only a subset of the data to my dataset.
2. Some of the text is too short. We have good variety of text lengths, with generally shorter texts than the wili2018 data, but overlapping with the wili2018 data. This is good because it will add to the variety of text lengths in my final dataset. The sentences from bld will also vary the dataset linguistically, since these are stand\-alone sentences with generally more informal speech represented than in the wili2018. However, some of the text is only 1 character in length. These sentences could be from languages with non\-Latin based alphabets.

I solved both of these problems in the data cleaning steps.

### Latin-Based Character Sets

I wanted to limiting my dataset to languages with a Latin alphabet, or an extension of the Latin alphabet, to make it easier to create my features. \(See further details on why in the [EDA notebook](EDA.ipynb#Limiting-to-Languages-with-Latin-Based-Alphabets)\). In order to do this, I calculated the ratio of Latin\-based characters of all the text in each language. Then I made a scatter plot for the ratios. \(See [the notebook](EDA.ipynb#Limiting-to-Languages-with-Latin-Based-Alphabets) for the scatter plot with hover text.\)

<img src="images/latin_ratio_scatter.png"   width="1193px"  height="423.667px"  style="object-fit:cover"/>

### Distribution of Data in Languages for BLD

I used a [pie chart](EDA.ipynb#Representing-Each-Language-Equally) to visualize the distribution, determining that it was necessary to sample data to evenly represent each language.

<img src="images/bld_language_distribution.png"   width="1188px"  height="453.833px"  style="object-fit:cover"/>



## Feature Engineering & Further EDA

I [created features](EDA.ipynb#Feature-Engineering) based on the text data that identify languages that don't rely on higher order organization or previously determined words. My features are character based, or based on character relationships within the text.

The features I created are:

1. Average congruent characters: The average length of sets of congruent characters separated by white space. This is an easy proxy for word length.
2. Index of dispersion, normalized by the length of the text: The ratio of variance of the number of characters in the text to the mean. This is a good representation of the relative frequency of characters.
3. Capital ratio: The ratio of capitalized alphabetic letters to total alphabetic letters. Languages may vary in the extent to which they use capital letters.
4. Top character ngrams \(bi/tri\-grams\): This feature is actually multiple features. I found the most predictive character bi\-grams and tri\-grams and add their counts in the text as features.

As I created these features, I scaled them to be in a similar range, on the order of 10^1.

### Feature 1

<img src="images/feature_1.png"   width="1188px"  />

### Feature 2

<img src="images/feature_2.png"   width="1188px"  />

### Feature 3

<img src="images/feature_3.png"   width="1188px"  />

### Visualization of First 3 Features

I created a [pair plot](EDA.ipynb#Visualizing-the-Feature-Data-So-Far) of the first 3 features, which showed me that there may be an outlier. 

<img src="images/pairplot_1.png"   width="613px"  height="593.117px"  style="object-fit:cover"/>

Some data points with extreme values seem to be junk data.

<img src="images/junk_data.png"   width="1280.1px"  height="627.567px"  style="object-fit:cover"/>

### N\-gram Features

The final features I made were character n\-gram features, where n is 2 or 3 \(bi\-grams/tri\-grams\). Creating these features included:

- Using CountVectorizer to count n\-grams for each text sample. Parameters: analyzer='char', ngram\_range=\(2,3\).
- Finding 500 of the top n\-gram features based on scoring with SelectKBest, using the "f\_classif" scorer. This scorer uses the ANOVA F\-value to estimate the degree of linear dependency between the label \(language class\) and the feature. It is a good scorer to use for classification using quantitative numeric data.

For more details about creating the n\-gram features, take a look at that [section of the notebook](EDA.ipynb#N-gram-Features).

In order to further select n\-grams, I used correlation data. The following image is a correlation heatmap of all 500 features.

<img src="images/correlation_heatmap_1.png"   width="1342.1px"  height="685.2px"  style="object-fit:cover"/>

This correlation heatmap shows me that there are some n\-grams that are highly correlated. I used a correlation value of 0.35 as my threshold in order to keep my feature set very limited, and to maintain a dataset with very low feature correlations. The following is the correlation heatmap after eliminating the correlated features.

<img src="images/correlation_heatmap_2.png"   width="1194px"  height="628px"  style="object-fit:cover"/>

I also did a [pair plot](EDA.ipynb#N-Gram-Features-Pair-Plot) with a limited set of n\-gram features.

<img src="images/pairplot_ngrams.png"   width="954px"  height="896.53px"  style="object-fit:cover"/>

Some of the things that this pair plot tells me are:

- Some of the n\-gram plots seem to have significant language clustering, which makes me think that they could be important for determining a language.
- This leads me to want to try decision tree based models because they would be able to easily split on these features.
- Because the n\-gram features are a limited set of integers, they may follow a pattern that logistic regression would also work well for modeling.



## Models

### Decision Tree Based Models

The reasons I tried decision tree based models were:

- To get interpretable output, including a decision tree and feature importances.
- Trees mimic the vertical aspect of language evolution. Languages historically branch off of root languages, so I would like to see if some of my features are modeled well through decision tree splitting.
- The pair plot data shows that some of the n\-grams may be particularly important in determining between languages.

I tuned the DecisionTreeClassifier model by testing a range of max\_depth hyperparameter values and chose the max\_depth value with the highest test accuracy. I then fit a RandomForestClassifier model with the same max\_depth value. I used the DecisionTreeClassifier model to output a tree and to compute feature importance to investigate which features were most significant to the model. \(These are further discussed in the [Results & Analysis](Main.ipynb#Results-&-Analysis) section of this notebook.\)

To see more details about these models, please view the [Modeling\_DecisionTree.ipynb](Modeling_DecisionTree.ipynb#Decision-Tree-Classifier) notebook.

### K\-Nearest Neighbors

The reason I tried the K\-Neighbors Classifier was to see if it could reasonably mimic the horizontal aspect of language evolution. Languages have not only evolved vertically through branching off, but also horizontally through cultural interaction. Language similarity may be clustered based on proximity to other cultural groups.

I determined which features to select for the KNeighborsClassifier model by fitting models with sets of features. I tested and determined that all of the features were contributing to the test accuracy of the model. Then, I tuned the n\_neighbors parameter by testing over a range.

To see more details, please view the [Modeling\_KNeighbors.ipynb](Modeling_KNeighbors.ipynb#K-Nearest-Neighbors) notebook.

### Logistic Regression

I chose to also do logistic regression because the pair plot data shows that because the n\-gram features are a limited set of integers, they may follow a pattern that logistic regression would also work well for modeling. I can also get good coefficient data from the model, which will allow me to understand more about feature importance in the context of this model.

I chose the "lbfgs" solver because it is ideal for a problem with many variables due to storing a limited version of the inverse hessian matrix; it can solve our high\-dimension problem quickly.  I tuned the C parameter by testing over a range scaled logarithmically.

To see more details, please view the [Modeling\_LogisticRegression.ipynb](Modeling_LogisticRegression.ipynb) notebook.



## Results & Analysis

The best models had the following accuracies and parameters:

| Model | Test Accuracy | Parameters |
| :--------------------- | :------------------ | :------------------------------------------------------------ |
| DecisionTreeClassifier | 0.3834418604651163 | max\_depth=39 |
| RandomForestClassifier | 0.4518604651162791 | max\_depth=39 |
| KNeighborsClassifier | 0.36153488372093023 | n\_neighbors=22 |
| LogisticRegression | 0.3991627906976744 | multi\_class='ovr', solver='lbfgs', C=best\_C, max\_iter=1000 |

The RandomForestClassifier performed the best. KNeighborsClassifier was the worst performer. The KNeighborsClassifier primarily focuses on local similarities, potentially missing complex interactions between features. If the data exhibits synergistic or antagonistic effects between features, LogisticRegression's feature interactions and RandomForest's multiple trees may be capturing these complexities better.

The following image shows the classification reports for the RandomForestClassifier, LogisticRegression, and KNeighbors models.  All classifications reports were sorted on f1\-score before charting because precision and recall are both important factors for the models.

<img src="images/classification_report_all.png"   width="946px"  height="890.03px"  style="object-fit:cover"/>

K\-Nearest Neighbors is the most balanced model, but has the lowest overall performance.  It may not be taking advantage of the nature of the n\-grams to improve its f1 scores and the scores may mostly reflect the basic performance of the other general features.  

LogisticRegression is the least balanced model.  While its overall accuracy is higher than K\-Nearest Neighbors, it may be that the nature of the algorithm is leading to very unbalanced results, as the n\-gram features are usually very biased towards a language or set of languages and logistic regression can overestimate the importance of features with a large impact on its predictions.

RandomForestClassifier has the best overall accuracy, and while it has some unbalanced target classes, their frequency is less than with LogisticRegression.  Decision Tree Classification appears best suited to take advantage of the n\-gram features to improve performance on individual classes.  The RandomForestClassifier's ensemble of decision trees are more likely to capture complex non\-linear relationships, like those between a significant number of n\-gram features.

### Confusion Matrices

Confusion matrices can be seen in the images folder for [RandomForestClassifier](images/confusionmatrix_rfc.png), [LogisticRegression](images/confusionmatrix_logreg.png), and [KNeighborsClassifier](images/confusionmatrix_knn.png). In general, the matrices show expected results: languages that are known to be related, either by physical proximity or due to large cultural influences such as colonization, are the most often confused with each other.

### Decision Tree Data Interpretation

An image of the decision tree can be found in [images/decision\_tree.png](images/decision_tree.png). The tree shows me that when n\-gram features are present in the data, it is usually highly predictive of a specific class or small set of class. When fewer n\-grams are present, the model relies heavily on the three statistical features to split and eventually reach it's conclusion.

The following image shows the Gini Importance of features in the best DecisionTreeClassifier model.

<img src="images/feature_importance.png"   width="1262px"  height="619.05px"  style="object-fit:cover"/>

The 3 statistical features \(avg\_cong\_chars, index\_of\_dispersion\_norm, and capital\_ratio\) are the most important features. Most of the n\-grams contribute little importance, which makes sense as they may only each be helpful in identifying a small subset of languages.



## Discussion and Conclusion

All models performed significantly better with some target classes than others, with the f1\-score for some languages being low in all models. This indicates to me that while the n\-gram features were effective at training toward some classes, they did not cover all classes.  There may not be enough n\-gram features to cover the languages being classified.

The RandomForestClassifier's performance indicates that the ideal model for this problem will be able to take advantage of features that strongly correlate with a language or set of languages.

To improve performance further, I would increase my dataset size and continue to engineer the n\-gram features. Further investigation would involve larger size n for n\-grams and expanding the number of n\-grams considered in a model. I would then use the decision trees to help measure feature importance and continue to iterate until I have features that represent all languages well. Another strategy that I could try is to use Principal Component Analysis on a large set of n\-gram features in order to try to get a smaller set of better features. I would also continue to investigate statistical character\-based features as they had the highest Gini Importance values in the DecisionTreeClassifier model.



## GitHub Repository

To use the repository, run cells in Data\_Cleaning.ipynb first.  This will create the parsed, cleaned dataset with generated features and store it as a .csv file.  Then run the cells in any of the Modeling notebooks to generate and test that model.

